In [ ]:
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, update, and_, text
from sqlalchemy.orm import sessionmaker

In [3]:
def connect_baltini():
    db_config = {
        'host': 'localhost',
        'port': '3307',
        'user': 'root',
        'password': 'root',
        'database': 'baltini'
    }
    connection_string = f"mysql+pymysql://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}"
    engine = create_engine(connection_string)
    return engine

In [5]:
date_filter = "CONCAT(DATE(NOW())-INTERVAL 1 DAY, ' 00:00:00')" #For daily automation
date_filter = "'2023-10-16 00:00:00'"

## 1a. Populate table product_duplicates using SCD1

FIRST DATA UPDATE

In [ ]:
conn = connect_baltini()
query = f"""SELECT   
                LOWER(REPLACE(CONCAT(SUBSTRING_INDEX(SUBSTRING_INDEX(p1.tags, 'ProductID: ', -1), ',', 1), '-', p1.gender, '-', p1.category), ' ', '')) AS title,
                MIN(p1.created_at) AS created_at,
                MIN(p1.created_at) AS updated_at 
            FROM 
                products p1
            GROUP BY 
                1
            HAVING 
                COUNT(title) > 1;
"""
products1 = pd.read_sql(query,conn)
print(len(products1))
products1

,title,created_at,updated_at
0,aaujw0212ot01gry0002-men-accessories,2023-10-16 15:18:26,2023-10-16 15:18:26
1,220433763030bubblegum-women-accessories,2023-10-16 16:20:30,2023-10-16 16:20:30
2,758065qzaj00900-women-clothing,2023-10-16 14:59:00,2023-10-16 14:59:00
3,aw23073sbblack-women-clothing,2023-10-16 15:04:21,2023-10-16 15:04:21
4,pm0001faa3x26m23ec-women-bags,2023-10-16 15:35:07,2023-10-16 15:35:07
...,...,...,...
35303,dg4422-women-accessories,2023-10-16 17:08:56,2023-10-16 17:08:56
35304,f3bwuahs147-men-clothing,2023-10-16 15:01:10,2023-10-16 15:01:10
35305,751477y7e631000-men-clothing,2023-10-16 15:46:17,2023-10-16 15:46:17
35306,ro02c1520srawm10693-women-clothing,2023-10-16 19:03:00,2023-10-16 19:03:00


In [12]:
products1.to_sql('product_duplicates',conn,if_exists='append',index=False)

35308

AUTOMATION START NOW

In [16]:
conn = connect_baltini()
query = f"""SELECT   
                LOWER(REPLACE(CONCAT(SUBSTRING_INDEX(SUBSTRING_INDEX(p1.tags, 'ProductID: ', -1), ',', 1), '-', p1.gender, '-', p1.category), ' ', '')) AS title_prod,
                MAX(p1.created_at) AS created_at,
                MAX(p1.created_at) AS updated_at 
            FROM 
                products p1
            GROUP BY 
                1
            HAVING 
                COUNT(title) > 1"""
products2 = pd.read_sql(query,conn)
print(len(products2))
products2

35308


,title_prod,created_at,updated_at
0,aaujw0212ot01gry0002-men-accessories,2023-10-16 16:25:34,2023-10-16 16:25:34
1,220433763030bubblegum-women-accessories,2023-10-16 16:52:54,2023-10-16 16:52:54
2,758065qzaj00900-women-clothing,2023-10-16 18:07:56,2023-10-16 18:07:56
3,aw23073sbblack-women-clothing,2023-10-16 17:49:09,2023-10-16 17:49:09
4,pm0001faa3x26m23ec-women-bags,2023-10-16 19:02:38,2023-10-16 19:02:38
...,...,...,...
35303,dg4422-women-accessories,2023-10-16 17:08:56,2023-10-16 17:08:56
35304,f3bwuahs147-men-clothing,2023-10-16 15:01:10,2023-10-16 15:01:10
35305,751477y7e631000-men-clothing,2023-10-16 16:24:16,2023-10-16 16:24:16
35306,ro02c1520srawm10693-women-clothing,2023-10-16 19:45:09,2023-10-16 19:45:09


In [ ]:
conn = connect_baltini()
query = """SELECT * FROM product_duplicates;"""
prod_dupe = pd.read_sql(query,conn)
prod_dupe
print(len(prod_dupe))
prod_dupe

35308


,id,title,created_at,updated_at
0,119e5027-affa-11ef-a8f0-0242ac110002,aaujw0212ot01gry0002-men-accessories,2023-10-16 15:18:26,2023-10-16 15:18:26
1,119ec239-affa-11ef-a8f0-0242ac110002,220433763030bubblegum-women-accessories,2023-10-16 16:20:30,2023-10-16 16:20:30
2,119ec683-affa-11ef-a8f0-0242ac110002,758065qzaj00900-women-clothing,2023-10-16 14:59:00,2023-10-16 14:59:00
3,119ec799-affa-11ef-a8f0-0242ac110002,aw23073sbblack-women-clothing,2023-10-16 15:04:21,2023-10-16 15:04:21
4,119ec874-affa-11ef-a8f0-0242ac110002,pm0001faa3x26m23ec-women-bags,2023-10-16 15:35:07,2023-10-16 15:35:07
...,...,...,...,...
35303,123b06be-affa-11ef-a8f0-0242ac110002,dg4422-women-accessories,2023-10-16 17:08:56,2023-10-16 17:08:56
35304,123b075c-affa-11ef-a8f0-0242ac110002,f3bwuahs147-men-clothing,2023-10-16 15:01:10,2023-10-16 15:01:10
35305,123b0803-affa-11ef-a8f0-0242ac110002,751477y7e631000-men-clothing,2023-10-16 15:46:17,2023-10-16 15:46:17
35306,123b0905-affa-11ef-a8f0-0242ac110002,ro02c1520srawm10693-women-clothing,2023-10-16 19:03:00,2023-10-16 19:03:00


In [18]:
joined_df = pd.merge(products2,prod_dupe,left_on='title_prod',right_on='title',how='left')
joined_df.head(5)

,title_prod,created_at_x,updated_at_x,id,title,created_at_y,updated_at_y
0,aaujw0212ot01gry0002-men-accessories,2023-10-16 16:25:34,2023-10-16 16:25:34,119e5027-affa-11ef-a8f0-0242ac110002,aaujw0212ot01gry0002-men-accessories,2023-10-16 15:18:26,2023-10-16 15:18:26
1,220433763030bubblegum-women-accessories,2023-10-16 16:52:54,2023-10-16 16:52:54,119ec239-affa-11ef-a8f0-0242ac110002,220433763030bubblegum-women-accessories,2023-10-16 16:20:30,2023-10-16 16:20:30
2,758065qzaj00900-women-clothing,2023-10-16 18:07:56,2023-10-16 18:07:56,119ec683-affa-11ef-a8f0-0242ac110002,758065qzaj00900-women-clothing,2023-10-16 14:59:00,2023-10-16 14:59:00
3,aw23073sbblack-women-clothing,2023-10-16 17:49:09,2023-10-16 17:49:09,119ec799-affa-11ef-a8f0-0242ac110002,aw23073sbblack-women-clothing,2023-10-16 15:04:21,2023-10-16 15:04:21
4,pm0001faa3x26m23ec-women-bags,2023-10-16 19:02:38,2023-10-16 19:02:38,119ec874-affa-11ef-a8f0-0242ac110002,pm0001faa3x26m23ec-women-bags,2023-10-16 15:35:07,2023-10-16 15:35:07


In [19]:
joined_df['insert_flag'] = joined_df[['title_prod','title']].apply(lambda x: 'Y' if (x[0]!=x[1] or pd.isnull(x[1])) else 'N', axis=1)
joined_df['update_flag'] = joined_df[['title_prod','title','updated_at_x','updated_at_y']].apply(lambda x: 'Y' if (x[0]==x[1] and x[2]!=x[3]) else 'N', axis=1)
joined_df

/var/folders/85/1jwp88lx7wn7_gdwt4ypryg00000gn/T/ipykernel_10602/2800839881.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  joined_df['insert_flag'] = joined_df[['title_prod','title']].apply(lambda x: 'Y' if (x[0]!=x[1] or pd.isnull(x[1])) else 'N', axis=1)
/var/folders/85/1jwp88lx7wn7_gdwt4ypryg00000gn/T/ipykernel_10602/2800839881.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  joined_df['update_flag'] = joined_df[['title_prod','title','updated_at_x','updated_at_y']].apply(lambda x: 'Y' if (x[0]==x[1] and x[2]!=x[3]) else 'N', axis=1)


,title_prod,created_at_x,updated_at_x,id,title,created_at_y,updated_at_y,insert_flag,update_flag
0,aaujw0212ot01gry0002-men-accessories,2023-10-16 16:25:34,2023-10-16 16:25:34,119e5027-affa-11ef-a8f0-0242ac110002,aaujw0212ot01gry0002-men-accessories,2023-10-16 15:18:26,2023-10-16 15:18:26,N,Y
1,220433763030bubblegum-women-accessories,2023-10-16 16:52:54,2023-10-16 16:52:54,119ec239-affa-11ef-a8f0-0242ac110002,220433763030bubblegum-women-accessories,2023-10-16 16:20:30,2023-10-16 16:20:30,N,Y
2,758065qzaj00900-women-clothing,2023-10-16 18:07:56,2023-10-16 18:07:56,119ec683-affa-11ef-a8f0-0242ac110002,758065qzaj00900-women-clothing,2023-10-16 14:59:00,2023-10-16 14:59:00,N,Y
3,aw23073sbblack-women-clothing,2023-10-16 17:49:09,2023-10-16 17:49:09,119ec799-affa-11ef-a8f0-0242ac110002,aw23073sbblack-women-clothing,2023-10-16 15:04:21,2023-10-16 15:04:21,N,Y
4,pm0001faa3x26m23ec-women-bags,2023-10-16 19:02:38,2023-10-16 19:02:38,119ec874-affa-11ef-a8f0-0242ac110002,pm0001faa3x26m23ec-women-bags,2023-10-16 15:35:07,2023-10-16 15:35:07,N,Y
...,...,...,...,...,...,...,...,...,...
35303,dg4422-women-accessories,2023-10-16 17:08:56,2023-10-16 17:08:56,123b06be-affa-11ef-a8f0-0242ac110002,dg4422-women-accessories,2023-10-16 17:08:56,2023-10-16 17:08:56,N,N
35304,f3bwuahs147-men-clothing,2023-10-16 15:01:10,2023-10-16 15:01:10,123b075c-affa-11ef-a8f0-0242ac110002,f3bwuahs147-men-clothing,2023-10-16 15:01:10,2023-10-16 15:01:10,N,N
35305,751477y7e631000-men-clothing,2023-10-16 16:24:16,2023-10-16 16:24:16,123b0803-affa-11ef-a8f0-0242ac110002,751477y7e631000-men-clothing,2023-10-16 15:46:17,2023-10-16 15:46:17,N,Y
35306,ro02c1520srawm10693-women-clothing,2023-10-16 19:45:09,2023-10-16 19:45:09,123b0905-affa-11ef-a8f0-0242ac110002,ro02c1520srawm10693-women-clothing,2023-10-16 19:03:00,2023-10-16 19:03:00,N,Y


In [20]:
insert_df = joined_df[joined_df['insert_flag'] == 'Y']
insert_df = insert_df[['title_prod','created_at_x','updated_at_x']]
insert_df.rename(columns = {'title_prod':'title','updated_at_x':'updated_at'},inplace=True)
insert_df

,title,created_at_x,updated_at


In [21]:
insert_df.to_sql('product_duplicates',conn,if_exists='append',index=False)

0

In [22]:
update_df = joined_df[joined_df['update_flag'] == 'Y']
update_df = update_df[['title_prod','created_at_x']]
update_df.rename(columns = {'title_prod':'title','created_at_x':'updated_at'},inplace=True)
update_df

,title,updated_at
0,aaujw0212ot01gry0002-men-accessories,2023-10-16 16:25:34
1,220433763030bubblegum-women-accessories,2023-10-16 16:52:54
2,758065qzaj00900-women-clothing,2023-10-16 18:07:56
3,aw23073sbblack-women-clothing,2023-10-16 17:49:09
4,pm0001faa3x26m23ec-women-bags,2023-10-16 19:02:38
...,...,...
35301,6d01503sph211000-women-clothing,2023-10-16 19:21:32
35302,170170281555-men-clothing,2023-10-16 19:29:18
35305,751477y7e631000-men-clothing,2023-10-16 16:24:16
35306,ro02c1520srawm10693-women-clothing,2023-10-16 19:45:09


In [23]:
Session = sessionmaker(bind=conn)
session = Session()

metadata = MetaData()
datatable = Table('product_duplicates', metadata, autoload_with=conn)

for ind, row in update_df.iterrows():
    upd = update(datatable).values({'updated_at': row.updated_at}).where(
        and_(datatable.c.title == row.title)
    )
    session.execute(upd)

session.commit()
session.close()


## 1b. Populate table product_duplicate_list

In [24]:
query = f"""SELECT 
pd.id product_duplicate_id,
external_id,
p.id product_id,
p.deleted_at,
p.created_at, 
p.updated_at 
FROM product_duplicates pd 
-- LEFT JOIN  products p  ON pd.title = p.title 
LEFT JOIN  products p  ON pd.title = LOWER(REPLACE(CONCAT(SUBSTRING_INDEX(SUBSTRING_INDEX(p.tags, 'ProductID: ', -1), ',', 1), '-', p.gender, '-', p.category), ' ', ''))
WHERE p.updated_at >= {date_filter}
"""
prod_dupe_list = pd.read_sql(query,conn)

prod_dupe_list.to_sql('product_duplicate_lists',conn, if_exists='append',index=False)
prod_dupe_list

,product_duplicate_id,external_id,product_id,deleted_at,created_at,updated_at
0,119e5027-affa-11ef-a8f0-0242ac110002,652d4d13150a8842ce6be273,0000636f-e333-40ab-90a8-926c9cd92681,NaT,2023-10-16 16:25:34,2023-10-16 16:25:34
1,119ec239-affa-11ef-a8f0-0242ac110002,652d5f2b27c9eb70a9e8cfb9,0000a5fe-5455-4869-afe5-337efeb7957d,NaT,2023-10-16 16:52:54,2023-10-16 16:52:54
2,119ec683-affa-11ef-a8f0-0242ac110002,652d6062cd33bf0a23ab1500,0000d0ee-5021-4243-979f-a0ad9f872afd,NaT,2023-10-16 17:48:26,2023-10-16 17:48:26
3,119ec799-affa-11ef-a8f0-0242ac110002,652d6067cd33bf0a23ab17f4,00014de9-848f-4cb0-99e2-07853503a2c5,NaT,2023-10-16 17:49:09,2023-10-16 17:49:09
4,119ec874-affa-11ef-a8f0-0242ac110002,652d5f2f27c9eb70a9e8ddaa,00049699-ed3c-4870-80ef-570f5b076b75,NaT,2023-10-16 16:55:50,2023-10-16 16:55:50
...,...,...,...,...,...,...
103498,11b55a13-affa-11ef-a8f0-0242ac110002,652d61452fa19d01408dd2f2,fffa098c-4714-42ea-a0ec-0752e1d59327,NaT,2023-10-16 18:09:04,2023-10-16 18:09:04
103499,11a2ab55-affa-11ef-a8f0-0242ac110002,652d4c6af0f8388b75894933,fffa5cd6-cdf6-4f3b-b669-0578401d881c,NaT,2023-10-16 15:53:20,2023-10-16 15:53:20
103500,12319d71-affa-11ef-a8f0-0242ac110002,652d606ecd33bf0a23ab1dae,fffeb587-7fbb-4a5e-9d43-6e1c559c0eae,NaT,2023-10-16 17:50:30,2023-10-16 17:50:30
103501,120db07c-affa-11ef-a8f0-0242ac110002,652d4b9fdc7a4cc2bfc8a6ac,fffebbd2-7238-4d91-afdf-ee0985846958,NaT,2023-10-16 15:21:53,2023-10-16 15:21:53


2. QUERY GROUP TITLE

In [25]:
query = """SELECT 
    pd.title  `Group Title`,
    COUNT(pdl.product_id)  AS `Product Count`, 
    MAX(pd.updated_at) AS `Updated At`
FROM 
    product_duplicates pd
JOIN 
    product_duplicate_lists pdl
    ON pd.id = pdl.product_duplicate_id 
GROUP BY 
    pd.title
ORDER BY 
    2 DESC;"""
result_df = pd.read_sql(query,conn)
result_df

,Group Title,Product Count,Updated At
0,rb3025-men-accessories,33,2023-10-16 17:04:45
1,rb2132-men-accessories,27,2023-10-16 17:04:48
2,omaa120f23jer0011001-men-clothing,25,2023-10-16 19:39:32
3,aulmll22-men-shoes,24,2023-10-16 19:39:32
4,omes005f23fab0036161-men-clothing,23,2023-10-16 19:38:57
...,...,...,...
35303,cogebh09191iaa-men-clothing,2,2023-10-16 19:23:04
35304,810134w1cv01000-women-shoes,2,2023-10-16 16:54:16
35305,me2262408c272-women-clothing,2,2023-10-16 19:24:30
35306,fd65ts1064si99j-men-clothing,2,2023-10-16 19:44:43
